In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import itertools
from itertools import product
import torch

print('mps: ', torch.backends.mps.is_available())

options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use the Metal backend
else:
    device = torch.device("cuda")  # Fallback to CPU

mps:  True


In [2]:
class BundledChoiceKP:
    def __init__(self,  num_agents , num_objects , lambda_star = None, num_simulations = 20, 
                 random_seed = 4,  sigma = 1, max_capacity = 100, generate_data = True):

        self.num_agents = num_agents
        self.num_objects = num_objects
        self.num_simulations = num_simulations


        self.random_seed = random_seed
        torch.manual_seed(self.random_seed)
        self.device = device
        self.agents_i = torch.arange(self.num_agents)
        self.agents_si = torch.kron(torch.ones(self.num_simulations, dtype = torch.int), torch.arange(self.num_agents))

        
        if lambda_star is not None:           
            # True parameters
            self.K_MOD = len(lambda_star)
            self.lambda_star_np = lambda_star
            self.lambda_star = torch.tensor(lambda_star, device=device, dtype=torch.float)

        else:                                 
            self.K_MOD = None
            self.lambda_star_np = None
            self.lambda_star = None

        if generate_data is True:
            ### Modular characteristics
            self.φ_i_j_k_numpy = np.random.normal(0, 1, (self.num_agents, self.num_objects, self.K_MOD))
            self.φ_i_j_k = torch.tensor(self.φ_i_j_k_numpy, device=device, dtype=torch.float) 

            # self.φ_i_j_k = torch.normal(0, 1, size=(self.num_agents, self.num_objects, self.K_MOD), 
            #                             device=device, dtype=torch.float)

            ### Knapsack Constraints
            self.weight_j = torch.randint(1, 100, (self.num_objects,), device=device, dtype = torch.int)
            self.capacity_i = torch.randint(1, max_capacity, (self.num_agents,), device=device, dtype=torch.int) 
        else:
            self.φ_i_j_k_numpy = None
            self.φ_i_j_k = None
            self.weight_j = None
            self.capacity_i = None
        
    
        ### Estimation
        self.φ_hat_k = None
        self.φ_hat_i_k = None
        self.value_LP = None

        self.eps_i_j = sigma * torch.normal(0, 1, size=(self.num_agents, self.num_objects), device=device, dtype=torch.float)
        self.eps_si_j = sigma * torch.normal(0, 1, size=(self.num_simulations * self.num_agents, self.num_objects), 
                                             device=device, dtype=torch.float)
        self.eps_s_i_j = self.eps_si_j.reshape(self.num_simulations, self.num_agents, self.num_objects)

    def load_data(self, characteristic_i_j, weight_j, capacity_i, matching_i_j):
        
        # Modular characteristics
        self.φ_i_j_k_numpy = characteristic_i_j
        self.φ_i_j_k  = torch.tensor(characteristic_i_j, dtype=torch.float32, device=self.device)
        self.K_MOD = self.φ_i_j_k.size(2)

        # Weights and capacities
        self.weight_j = torch.tensor(weight_j, dtype=torch.int64, device=self.device)
        self.capacity_i = torch.tensor(capacity_i, dtype=torch.int64, device=self.device)
       
        # Observed moments from matching
        self.φ_hat_k = (self.φ_i_j_k_numpy * matching_i_j[:,:,None]).sum((0,1))

    

In [3]:
def linear_knapsack(self, idx, lambda_k , max_weight = None, p_j = None, eps_i_j = None, return_value = False):
    
    ### Compute the values
    profit_i_j = self.φ_i_j_k[idx]  @  lambda_k 

    if p_j is not None:
        profit_i_j -= p_j.unsqueeze(0)
    if eps_i_j is not None:
        profit_i_j += eps_i_j

    if max_weight is None:
        max_weight = int(self.capacity_i[idx].max())
    
    ### Fill in the DP table
    value_i_j_w = torch.zeros((len(idx), self.num_objects +1, max_weight +1) ,device=device, dtype=torch.float)
    weight_states = torch.arange(max_weight + 1, device = device)

    # print type of weight_states
    for j in range(self.num_objects):
        value_i_j_w[:, j+1, :] = torch.where(self.weight_j[j] <= weight_states, 
                                    torch.maximum(profit_i_j[:, j].unsqueeze(1) + value_i_j_w[:, j, weight_states - self.weight_j[j]],
                                                value_i_j_w[:, j, :]), 
                                        value_i_j_w[:, j, :])

    ### Backtrack to find the items
    residual_weight = self.capacity_i[idx]
    B_i_j_star = torch.zeros((len(idx), self.num_objects), device=device, dtype= bool)

    for j in range(self.num_objects,0,-1):

        pick_j = (value_i_j_w[torch.arange(len(idx)), j, residual_weight] > 
                    value_i_j_w[torch.arange(len(idx)), j-1, residual_weight])
        
        B_i_j_star[:, j-1] = pick_j
        residual_weight -= pick_j * self.weight_j[j-1]

    if return_value:
        return B_i_j_star, value_i_j_w[torch.arange(len(idx)), -1 , self.capacity_i[idx]]
    else:
        return B_i_j_star 

BundledChoiceKP.linear_knapsack = linear_knapsack

### Estimate assignment

In [11]:
def estimate_GMM_matching(self, max_iters = 100, tol = 1e-2 ):

                ### Initialize
                env = gp.Env(params=options) 
                model = gp.Model(env=env) 
                # Create variables
                lambda_k = model.addVars(self.K_MOD , lb= 0, ub = 1e6 , name="parameters")
                u_si = model.addVars(self.num_simulations * self.num_agents, lb = 0, ub = GRB.INFINITY , name="utilities")
                p_j = model.addVars(self.num_objects, lb = 0, ub = GRB.INFINITY , name="prices")

                # Set objective and initial constraints
                model.setObjective(gp.quicksum( self.φ_hat_k[k] * lambda_k[k] for k in range(self.K_MOD ))
                                    - (1 / self.num_simulations) * u_si.sum() - p_j.sum(), GRB.MAXIMIZE)

                # Solve master problem 
                model.setParam('OutputFlag', 0)
                model.optimize()
                solution_master_pb = np.array(model.x)

                iter = 0
                while iter < max_iters:
                    print('################')
                    print(f"ITER: {iter}, parameters: {solution_master_pb[:self.K_MOD]}")
            
                    ### Pricing problem
                    lambda_k_iter = torch.tensor(solution_master_pb[:self.K_MOD], device= device , dtype=torch.float32)
                    p_j_iter = torch.tensor(solution_master_pb[-self.num_objects:], device= device , dtype=torch.float32)

                    B_star_si, val_si = self.linear_knapsack(
                                                                self.agents_si, 
                                                                lambda_k_iter, 
                                                                eps_i_j = self.eps_si_j,
                                                                p_j = p_j_iter,
                                                                return_value = True
                                                                )

                    ### Stop if certificate holds
                    certificate_i = val_si.cpu().numpy() - solution_master_pb[self.K_MOD: - self.num_objects]
                    max_certificate = np.max(certificate_i)

                    print("Value of LP:   ", model.objVal)
                    print(f"Reduced cost: {max_certificate}")
                    if max_certificate < tol:
                            print('#############################################')
                            print("Solution:       ", solution_master_pb[:self.K_MOD].round(3))
                            if self.lambda_star_np is not None:
                                print("True parameters:", self.lambda_star_np)
                            print('Number of iterations: ', iter)
                            print('#############################################')
                            return solution_master_pb , np.array(model.pi)

                    ### Master problem
                    # Add constraints
                    B_star_si = B_star_si.cpu().numpy()
                    φ_si_k_star = (self.φ_i_j_k_numpy[self.agents_si] * B_star_si[:,:,None]).sum(1)
                    
                    model.addConstrs(
                                    (u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])
                                    >= gp.quicksum((φ_si_k_star[si, k]) * lambda_k[k]  for k in range(self.K_MOD))
                                    + self.eps_si_j[si, B_star_si[si]].sum()
                                    for si in range(self.num_simulations * self.num_agents))
                                    )
                    
                    # Solve master problem
                    lambda_k.start = solution_master_pb[ : self.K_MOD]
                    p_j.start = p_j_iter.cpu().numpy()
                    u_si.start = val_si.cpu().numpy() 

                    model.optimize()
                    solution_master_pb = np.array(model.x)

                    iter += 1
        

BundledChoiceKP.estimate_GMM_matching = estimate_GMM_matching

In [13]:
def estimate_GMM_matching_batched(self, max_iters = 100, tol = 1e-2 ):
                env = gp.Env(params=options) 
                model = gp.Model(env=env) 
                ### Initialize
                # Create variables
                lambda_k = model.addVars(self.K_MOD , lb= -1e6, ub = 1e6 , name="parameters")
                u_si = model.addVars(self.num_simulations * self.num_agents, lb = 0, ub = GRB.INFINITY , name="utilities")
                p_j = model.addVars(self.num_objects, lb = 0, ub = GRB.INFINITY , name="prices")

                # Set objective and initial constraints
                model.setObjective(gp.quicksum( self.φ_hat_k[k] * lambda_k[k] for k in range(self.K_MOD ))
                                    - (1 / self.num_simulations) * u_si.sum() - p_j.sum(), GRB.MAXIMIZE)

                # Solve master problem 
                model.setParam('OutputFlag', 0)
                model.optimize()
                solution_master_pb = np.array(model.x)

                iter = 0
                while iter < max_iters:
                    print('################')
                    print(f"ITER: {iter}, parameters: {solution_master_pb[:self.K_MOD]}")
            
                    ### Pricing problem
                    lambda_k_iter = torch.tensor(solution_master_pb[:self.K_MOD], device= device , dtype=torch.float32)
                    p_j_iter = torch.tensor(solution_master_pb[-self.num_objects:], device= device , dtype=torch.float32)
                    B_star_si = []
                    val_si = []
                    for simul in range(self.num_simulations):
                        B_star_i, val_i = self.linear_knapsack(
                                                                    self.agents_i, 
                                                                    lambda_k_iter, 
                                                                    eps_i_j = self.eps_s_i_j[simul],
                                                                    p_j = p_j_iter,
                                                                    return_value = True
                                                                    )
            
                        B_star_si.append(B_star_i)
                        val_si.append(val_i)

                    B_star_si = torch.cat(B_star_si, dim= 0)
                    val_si = torch.cat(val_si, dim= 0)

                    ### Stop if certificate holds
                    certificate_i = val_si.cpu().numpy() - solution_master_pb[self.K_MOD: - self.num_objects]
                    max_certificate = np.max(certificate_i)

                    print("Value of LP:   ", model.objVal)
                    print(f"Reduced cost: {max_certificate}")
                    if max_certificate < tol:
                            print('#############################################')
                            print("Solution:       ", solution_master_pb[:self.K_MOD].round(3))
                            if self.lambda_star_np is not None:
                                print("True parameters:", self.lambda_star_np)
                            print('Number of iterations: ', iter)
                            print('#############################################')
                            return solution_master_pb , np.array(model.pi)

                    ### Master problem
                    # Add constraints
                    B_star_si = B_star_si.cpu().numpy()
                    φ_si_k_star = (self.φ_i_j_k_numpy[self.agents_si] * B_star_si[:,:,None]).sum(1)
                    
                    model.addConstrs(
                                    (u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])
                                    >= gp.quicksum((φ_si_k_star[si, k]) * lambda_k[k]  for k in range(self.K_MOD))
                                    + self.eps_si_j[si, B_star_si[si]].sum()
                                    for si in range(self.num_simulations * self.num_agents))
                                    )
                    
                    # Solve master problem
                    lambda_k.start = solution_master_pb[ : self.K_MOD]
                    p_j.start = p_j_iter.cpu().numpy()
                    u_si.start = val_si.cpu().numpy() 

                    model.optimize()
                    solution_master_pb = np.array(model.x)

                    iter += 1
        

BundledChoiceKP.estimate_GMM_matching_batched = estimate_GMM_matching_batched